<a href="https://colab.research.google.com/github/SpyroRock/U-net-reconstruction/blob/master/Reconstrunction_algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/SpyroRock/U-net-reconstruction
%cd U-net-reconstruction

Cloning into 'U-net-reconstruction'...
remote: Enumerating objects: 198, done.
remote: Counting objects: 100% (62/62), done.
remote: Compressing objects: 100% (45/45), done.
remote: Total 198 (delta 27), reused 49 (delta 17), pack-reused 136
Receiving objects: 100% (198/198), 226.69 MiB | 19.09 MiB/s, done.
Resolving deltas: 100% (99/99), done.
/content/U-net-reconstruction


In [2]:
!pip install -r requirements.txt

     |████████████████████████████████| 92.6MB 41kB/s 
     |████████████████████████████████| 317kB 44.2MB/s 
     |████████████████████████████████| 3.2MB 34.3MB/s 
     |████████████████████████████████| 368kB 38.1MB/s 
     |████████████████████████████████| 51kB 7.0MB/s 
     |████████████████████████████████| 3.3MB 32.5MB/s 
  Created wheel for keras-segmentation: filename=keras_segmentation-0.3.0-cp37-none-any.whl size=29072 sha256=79a581414a2e5c1fe9b15384c9d710ad941993058d71733cd613d79349575adf
  Stored in directory: /root/.cache/pip/wheels/d0/bb/c0/6aad88b38f6e46db048bed4cccb904a5897055a8ab6fbd4dfc
Successfully built keras-segmentation
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: tensorboard 2.4.1
    Uninstalling tensorboard-2.4.1:
      Successfully uninstalled tensorboard-2.4.1
  Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimato

In [3]:
%cd /content/U-net-reconstruction/

/content/U-net-reconstruction


In [4]:
from model_structure import create_model
from U_net_mix import unet_model
from unet_model_construction import get_unet
import keras
from keras.models import Model
from keras.layers import Input
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
from keras import backend as K
from keras.callbacks import ModelCheckpoint
import numpy as np
from numpy import load, save
from sklearn.model_selection import train_test_split
import pickle

Using TensorFlow backend.
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarn

In [5]:
img_height = 128
img_width = 128

img_height_test = 128
img_width_test = 128

speckle_data = load('speckle_array_case0.npy')
print(speckle_data.shape)

speckle_labels = load('symbol_array_case0.npy')
print(speckle_labels.shape)

(258, 128, 128)
(258, 128, 128)


In [9]:
X_train, X_test, y_train, y_test = train_test_split(speckle_data, speckle_labels, test_size=0.1, random_state=42)

# plt.imshow(X_train[0], cmap='gray')
# plt.show()
# plt.imshow(y_train[0], cmap='gray')
# plt.show()

X_train = X_train.reshape(-1, img_height, img_width, 1)
X_test = X_test.reshape(-1, img_height, img_width, 1)
input_shape = (img_height, img_width, 1)

y_train = y_train.reshape(-1, img_height_test, img_width_test, 1)
y_test = y_test.reshape(-1, img_height_test, img_width_test, 1)
input_shape_test = (img_height_test, img_width_test, 1)

In [10]:
# define the checkpoint
filepath = "content/U-net-reconstruction/model.h5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)

reconstruction = get_unet(input_shape, n_filters = 16, dropout = 0.1, batchnorm = True)

reconstruction.fit(X_train, y_train, 
                   batch_size = 10, 
                   epochs = 500, 
                   verbose = 1, 
                   validation_data = (X_test, y_test),
                   callbacks=callbacks_list) # Data on which to evaluate the loss and any model metrics at the end of each epoch. 
                                                       # The model will not be trained on this data. 
                                                       # This could be a list (x_val, y_val) or a list (x_val, y_val, val_sample_weights). 
                                                       # validation_data will override validation_split.

score = reconstruction.evaluate(X_test, y_test, verbose = 0)

print('Test loss:', score[0])
print('Test acuracy:', score[1]) 

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 128, 128, 1)  0                                            
__________________________________________________________________________________________________
conv2d_40 (Conv2D)              (None, 128, 128, 16) 160         input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_38 (BatchNo (None, 128, 128, 16) 64          conv2d_40[0][0]                  
__________________________________________________________________________________________________
activation_38 (Activation)      (None, 128, 128, 16) 0           batch_normalization_38[0][0]     
__________________________________________________________________________________________________
max_poolin

KeyboardInterrupt: ignored